<a href="https://colab.research.google.com/github/codedyasai/Python_MachineLearning/blob/main/19_%EC%8B%A0%EC%9A%A9%EC%B9%B4%EB%93%9C_%EC%82%AC%EA%B8%B0_%EA%B2%80%EC%B6%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import display
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 음수표현 라이브러리
plt.rcParams['axes.unicode_minus'] = False

# 경고무시
import warnings
warnings.filterwarnings("ignore")

# 매직명령어 : 시각화 결과가 노트북에 포함되도록
%matplotlib inline

#### 불균형 데이터(Imbalanced Data) 처리를 위한 샘플링 기법

**불균형 데이터란?**
- 정상 범주의 관측치 수와 이상 범주의 관측치 수가 현저히 차이가 나는 데이터

- 문제점: 정상을 정확히 분류하는 것과 이상을 정확히 분류하는 것 중 일반적으로 이상을 정확히 분류하는 것이 더 중요하다.<br> 보통 이상 데이터가 target이 되는 경우가 더 많기 때문이다.
- 불균형한 데이터 세트는 이상 데이터를 정확히 찾아내지 못할 수 있다는 문제점이 존재한다.

**데이터를 조정해서 불균형 데이터를 해결하는 샘플링 기법들**

1. 언더 샘플링: 다수 범주의 데이터를 소수 범주의 데이터 수에 맞게 줄이는 샘플 방식
  - Random Sampling: 다수의 범주에서 무작위로 샘플링하는 것
  - Tomek Links: 두 범주 사이를 탐지하고 정리를 통해 부정확한 분류경계선을 방지하는 방법
  - CNN Rule
  - One Sided Selection: Tomek Links + CNN Rule 결합한 방식

2. 오버 샘플링: 소수 범주의 데이터 수를 다수 범주의 데이터 수와 비슷해지도록 증가시키는 방법
  - Resampling: 소수 범주의 데이터 수를 다수 범주의 데이터 수와 비슷해지도록 증가시키는 방법
  - SMOTE: 소수 범주에서 가상의 데이터를 생성하는 방법
  - GAN

**피처 엔지니어링(특성 공학)**
- Log 변환
- IQR(Inter Quantile Range) = Q3 - Q1, 이상치
  value > Q3 + 1.5 * IQR
  value < Q1 - 1.5 * IQR

- 언더 샘플링, 오버 샘플링


## 데이터 로딩 및 탐색

In [ ]:
card_df = pd.read_csv('creditcard.csv')
card_df

FileNotFoundError: ignored

In [ ]:
card_df.shape

- 이상 거래 판단할 관련 데이터셋
- 이상 거래는 카드값을 지불하지 않을 의도를 가지고서 결제를 하거나, 도난된 카드를 가지고 결제를 하는 등의 거래를 말한다.
- 종속변수: 이상거래 여부
- 알고리즘 종류: 분류
- 평가지표: 정확도, 혼동행렬, 분류 리포트 등

## 전처리

In [ ]:
card_df.drop('Time', axis= 1, inplace= True)

In [ ]:
card_df = card_df.dropna(subset=['Class'])


In [ ]:
card_df.Class.unique()

In [ ]:
card_df.Class.value_counts()

In [ ]:
x = card_df.iloc[:, :-1]
y = card_df.iloc[:, -1]

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size= 0.2, stratify= y, random_state= 0)

In [ ]:
# target에 대한 학습셋과 테스트셋의 비율
print(y_train.value_counts()/ y_train.shape[0] * 100)
print('_' * 50)
print(y_train.value_counts()/ y_train.shape[0] * 100)

# 모델링

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(x_train, y_train)

lr_pred = lr.predict(x_test)
lr_pred_proba = lr.predict_proba(x_test)[:, 1]

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.metrics import f1_score, confusion_matrix, roc_curve, precision_recall_curve

# get(실제값, 예측값, 예측확률)
def get(y_test, pred=None, pred_proba=None):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    f1 = f1_score(y_test, pred)
    roc_auc = roc_auc_score(y_test, pred_proba)

    print('오차 행렬(혼돈 행렬)')
    print(confusion)

    print(f'정확도:{accuracy:.4f}, 정밀도:{precision:.4f}, 재현율:{recall:.4f}, F1:{f1:.4f}, AUC:{roc_auc:.4f}')

In [ ]:
get(y_test, lr_pred, lr_pred_proba)

In [ ]:
# amount라는 피처에 대한 스케일링이 필요하다.

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
amount_n = scaler.fit_transform(card_df['Amount'].values.reshape(-1, 1))

card_df.insert(0, 'AmountScaled', amount_n)
card_df.drop(['Amount'], axis= 1, inplace= True)
card_df

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size= 0.2, stratify= y, random_state= 0)

lr = LogisticRegression()
lr.fit(x_train, y_train)

lr_pred = lr.predict(x_test)
lr_pred_proba = lr.predict_proba(x_test)[:, 1]

get(y_test, lr_pred, lr_pred_proba)

- **Amount의 로그변환** - 왜곡된 분포도를 가진 데이터를 정규분포에 가깝게 변환해주는 것

In [ ]:
# 데이터 사본 생성
card = pd.read_csv('creditcard.csv')

In [ ]:
card.describe()

In [ ]:
amount_n = np.log1p(card['Amount'])
card.drop(['Time', 'Amount'], axis= 1, inplace= True)
card.insert(0, 'AmountScaled', amount_n)
card.head()

In [ ]:
x = card.iloc[:, :-1]
y = card.iloc[:, -1]

x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                    test_size= 0.2, stratify= y, random_state= 0)

lr = LogisticRegression()
lr.fit(x_train, y_train)

lr_pred = lr.predict(x_test)
lr_pred_proba = lr.predict_proba(x_test)[:, 1]

get(y_test, lr_pred, lr_pred_proba)

## SMOTE 오버 샘플링 적용 후 모델 학습

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import SMOTE

smote = SMOTE()

x_train_over, y_train_over = smote.fit_resample(x_train, y_train)

print('SMOTE 적용 전 학습데이터/타깃데이터세트: ', x_train.shape, y_train.shape)
print('SMOTE 적용 후 학습데이터/타깃데이터세트: ', x_train_over.shape, y_train_over.shape)

In [ ]:
lr = LogisticRegression()
lr.fit(x_train_over, y_train_over)

lr_pred = lr.predict(x_test)
lr_pred_proba = lr.predict_proba(x_test)[:, 1]

get(y_test, lr_pred, lr_pred_proba)